In [1]:
!pip install SentencePiece
!pip install transformers
!pip install rich
!pip install evaluate
!pip install SceneGraphParser
!python -m spacy download en  
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# global var set
import transformers
# model info, change as needed
batch_size = 26
num_epochs = 20

model_checkpoint = "mrm8488/t5-base-finetuned-common_gen"
# model_checkpoint = "t5-base"
# metric_name = "f1"
fileTag = "original-plutchik-v1"
# fileTag = "clean-v1"

In [2]:
import pandas as pd
from pandas import DataFrame
!git clone https://github.com/Stry233/A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability.git
trainDatasetOriginal = pd.read_csv(f'./A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability/data/csv_version/dev/emotion/allcharlinepairs-{fileTag}.csv')
testDatasetOriginal = pd.read_csv(f'./A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability/data/csv_version/test/emotion/allcharlinepairs-{fileTag}.csv')
trainDatasetOriginal = pd.concat([trainDatasetOriginal, testDatasetOriginal]).reset_index(drop=True) # shuffle and combine everything, ratio issue fix here!

fatal: destination path 'A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability' already exists and is not an empty directory.


In [6]:
!dir

A-Emotion-Data-Cleaning-Pipeline-for-Enhancing-Data-Reliability  sample_data
drive


In [7]:
trainDatasetOriginal

,Unnamed: 0,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
0,0,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,1,I (myself),ann0,NaN,I began making fish curry for my boyfriend and I.,yes,"['Joy and excited to be making the food', 'con...","{'joy': 1, 'trust': 0, 'fear': 0, 'surprise': ..."
1,1,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,2,I (myself),ann0,I began making fish curry for my boyfriend and I.,I decided not to read a recipe since I've made...,yes,"[""I decided not to read a recipe since I've ma...","{'joy': 1, 'trust': 2, 'fear': 0, 'surprise': ..."
2,2,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,3,I (myself),ann0,I began making fish curry for my boyfriend and...,I let the curry sit before tasting.,yes,"['anxious', 'confident', 'positive', 'none']","{'joy': 1, 'trust': 1, 'fear': 0, 'surprise': ..."
3,3,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,4,I (myself),ann0,I began making fish curry for my boyfriend and...,"When it was time to taste, I was disgusted.",yes,"['When it was time to taste', 'I was disgusted...","{'joy': 0, 'trust': 0, 'fear': 1, 'surprise': ..."
4,4,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,5,I (myself),ann0,I began making fish curry for my boyfriend and...,I accidentally used a whole garlic instead of ...,yes,['I accidentally used a whole garlic instead o...,"{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ..."
...,...,...,...,...,...,...,...,...,...,...
24260,11845,9b53627e-41e5-47e6-9c6c-74d485572958,1,Rosemary,none,NaN,Rosemary was eating a snack.,no,['none'],"{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ..."
24261,11846,9b53627e-41e5-47e6-9c6c-74d485572958,2,Rosemary,ann0,Rosemary was eating a snack.,She was at a party that had amazing food set out.,yes,"['glad', 'satisfied', 'excited']","{'joy': 3, 'trust': 2, 'fear': 0, 'surprise': ..."
24262,11847,9b53627e-41e5-47e6-9c6c-74d485572958,3,Rosemary,ann0,Rosemary was eating a snack.|She was at a part...,She had been dieting for weeks and was starving.,yes,"['Agitated', 'deprived', 'famished']","{'joy': 1, 'trust': 1, 'fear': 0, 'surprise': ..."
24263,11848,9b53627e-41e5-47e6-9c6c-74d485572958,4,Rosemary,ann0,Rosemary was eating a snack.|She was at a part...,"After she stuffed herself, she regretted it.",yes,"['gross', 'regret']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ..."


# Data preprocessing
## As an example
### input:
Based on the following sentence <...>. The next sentence with the following emotion <...> should be
### output
<...>

In [5]:
import ast
import sng_parser
def getKeyword(inputSentence):
  graph = sng_parser.parse(inputSentence)
  return (", ".join((key.lower()).replace("a ", "").replace("the ", "").replace("an ", "")
      for key in [x['span'] for x in graph['entities']])).replace("i,", "I,")
def parseRemapPlutchik(emotionList):
    emotionListDict = ast.literal_eval(emotionList)
    report = ""
    for emoLabel in ['joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust', 'anger', 'anticipation']:
      report += "" if emotionListDict[emoLabel] == 0 else f"{emoLabel}, "
    return report.strip(", ")
    
def getPreviousSentenceWPrompt(inputDf):
    generalEntity = "it"
    previousSentenceFilter = inputDf[[True if index != len(inputDf['linenum'])-1 and inputDf.iloc[index]['linenum'] < inputDf.iloc[index+1]['linenum'] else False for index, linenum in enumerate(inputDf['linenum'])]] # remove last sentence of each story
    nextEmotionFilter = inputDf[[True if index != 0 and inputDf.iloc[index]['linenum'] > inputDf.iloc[index-1]['linenum'] else False for index, linenum in enumerate(inputDf['linenum'])]] # remove the first sentence of each story
    return [(f"Generate next sentence that makes reader feels {parseRemapPlutchik(nextEmotion[1]['plutchik'])}."
        +f"<extra_id_0>KEYWORD: {getKeyword(nextEmotion[1]['sentence'])}"
        +f"<extra_id_1>CONTEXT: {nextEmotion[1]['context'].replace('|', '')}")
        for previousSentence, nextEmotion in zip(previousSentenceFilter.iterrows(), nextEmotionFilter.iterrows())]

def getNextSentence(inputDf):
    return inputDf[[True if index != 0 and inputDf.iloc[index]['linenum'] > inputDf.iloc[index-1]['linenum'] else False for index, linenum in enumerate(inputDf['linenum'])]]['sentence']


In [ ]:
processedDataTrain = DataFrame({'question': getPreviousSentenceWPrompt(trainDatasetOriginal), "answer":getNextSentence(trainDatasetOriginal)}).reset_index(drop=True)
processedDataTest = DataFrame({'question': getPreviousSentenceWPrompt(testDatasetOriginal), "answer":getNextSentence(testDatasetOriginal)}).reset_index(drop=True)

In [ ]:
processedDataTest
processedDataTest.style.set_properties(subset=['question'], **{'width-min': '300px'})

In [ ]:
processedDataTrain.to_csv(f'./genV2-{fileTag}-train.csv', index=False)
processedDataTrain

In [12]:
processedDataTest.to_csv(f'./genV2-{fileTag}-test.csv', index=False)
processedDataTest

,question,answer
0,Generate next sentence that makes reader feels...,He dropped one on the floor by accident.
1,Generate next sentence that makes reader feels...,As a joke he pretended that it was a soccer ball.
2,Generate next sentence that makes reader feels...,He kicked the orange across the kitchen.
3,Generate next sentence that makes reader feels...,It landed in a pot and he cheered!
4,Generate next sentence that makes reader feels...,She had been preparing for hours.
...,...,...
9475,Generate next sentence that makes reader feels...,Carol had to move around in the dark.
9476,Generate next sentence that makes reader feels...,She was at a party that had amazing food set out.
9477,Generate next sentence that makes reader feels...,She had been dieting for weeks and was starving.
9478,Generate next sentence that makes reader feels...,"After she stuffed herself, she regretted it."


# Start Fine tuning

In [3]:
import pandas as pd

dfTrain = pd.read_csv(f'./genV2-{fileTag}-train.csv')
dfTest = pd.read_csv(f'./genV2-{fileTag}-test.csv')
dfAll = pd.concat([dfTest, dfTrain]).sample(frac=1).reset_index(drop=True) # shuffle and combine everything, ratio issue fix here!

In [4]:
dfAll

,question,answer
0,Generate next sentence that makes reader feels...,My little sister started swinging her net at t...
1,Generate next sentence that makes reader feels...,"Sadly, she cannot drink Diet Coke while pregnant."
2,Generate next sentence that makes reader feels...,"The kids loved it, to her shock!"
3,Generate next sentence that makes reader feels...,One day he and a friend blew up their lab.
4,Generate next sentence that makes reader feels...,He ordered a fancy new king size bed online.
...,...,...
28887,Generate next sentence that makes reader feels...,Then I tried 75 pounds.
28888,Generate next sentence that makes reader feels...,Cathy was so excited that she would be teachin...
28889,Generate next sentence that makes reader feels...,We met on the playground while our parents pic...
28890,Generate next sentence that makes reader feels...,"Alas, there was a hole in the suit."


In [5]:
"""
refer to Shivanand Roy 的visualization code，
visualize dataframe，optional
"""

# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
    """
    display dataframe in ASCII format
    """

    console=Console() 
    table = Table(Column("source_text (Question)", justify="center" ), Column("target_text (Answer)", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)


training_logger = Table(Column("Epoch", justify="center" ), 
             Column("Steps", justify="center"),
             Column("Loss", justify="center"), 
             title="Training Status",pad_edge=False, box=box.ASCII)


In [7]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [8]:
class NextSentenceDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
      self.tokenizer = tokenizer
      self.data = dataframe
      self.source_len = source_len
      self.summ_len = target_len
      self.target_text = self.data[target_text]
      self.source_text = self.data[source_text]

  def __len__(self):
      return len(self.target_text)

  def __getitem__(self, index):
      source_text = str(self.source_text[index])
      target_text = str(self.target_text[index])

      #cleaning data so as to ensure data is in string type
      source_text = ' '.join(source_text.split())
      target_text = ' '.join(target_text.split())

      source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
      target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

      source_ids = source['input_ids'].squeeze()
      source_mask = source['attention_mask'].squeeze()
      target_ids = target['input_ids'].squeeze()
      target_mask = target['attention_mask'].squeeze()

      return {
          'source_ids': source_ids.to(dtype=torch.long), 
          'source_mask': source_mask.to(dtype=torch.long), 
          'target_ids': target_ids.to(dtype=torch.long),
          'target_ids_y': target_ids.to(dtype=torch.long)
      }

In [9]:
def train(epoch, tokenizer, model, device, loader, optimizer):
  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%1000==0:
        training_logger.add_row(str(epoch), str(_), str(loss))
        console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [10]:
import nltk
bleu = []
def validate(epoch, tokenizer, model, device, loader):
  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
    for _, data in enumerate(loader, 0):
      y = data['target_ids'].to(device, dtype = torch.long)
      ids = data['source_ids'].to(device, dtype = torch.long)
      mask = data['source_mask'].to(device, dtype = torch.long)

      # generated_ids = model.generate(
      #     input_ids = ids,
      #     attention_mask = mask, 
      #     max_length=150, 
      #     num_beams=2,
      #     repetition_penalty=2.5, 
      #     length_penalty=1.0, 
      #     early_stopping=True
      # )
      generated_ids = model.generate( 
                input_ids = ids, 
                attention_mask = mask, 
                max_length=150, 
                top_k=3, 
                repetition_penalty=2.5,
                length_penalty=1.0, 
                early_stopping=True)
      preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
      target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
      BLEUscore = nltk.translate.bleu_score.sentence_bleu(target, preds)
      if _%10==0:
          console.print(f'Completed {_}')

      predictions.extend(preds)
      actuals.extend(target)
      bleu.append(BLEUscore)
  return predictions, actuals

In [11]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):  
    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"]) # pytorch random seed
    np.random.seed(model_params["SEED"]) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text,target_text]]
    display_df(dataframe.head(2))


    """ !!!ATTENTION PART!!!"""
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
    train_size = 1
    train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
    val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = NextSentenceDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
    val_set = NextSentenceDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': model_params["TRAIN_BATCH_SIZE"],
        'shuffle': True,
        'num_workers': 0
        }


    val_params = {
        'batch_size': model_params["VALID_BATCH_SIZE"],
        'shuffle': False,
        'num_workers': 0
        }


    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params = model.parameters(), lr=model_params["LEARNING_RATE"])


    # Training loop
    console.log(f'[Initiating Fine Tuning]...\n')

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)
      
    # console.log(f"[Saving Model]...\n")
    #Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)


    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
    console.save_text(os.path.join(output_dir,'logs.txt'))

    console.log(f"[Validation Completed.]\n")
    console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
    console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [12]:
model_params={
  "MODEL": model_checkpoint,          # model_type: t5-base/t5-large
  "TRAIN_BATCH_SIZE":batch_size,            # training batch size
  "VALID_BATCH_SIZE":1,             # validation batch size
  "TRAIN_EPOCHS":num_epochs,               # number of training epochs
  "VAL_EPOCHS":1,                # number of validation epochs
  "LEARNING_RATE":5e-4,             # learning rate
  "MAX_SOURCE_TEXT_LENGTH":512,         # max length of source text
  "MAX_TARGET_TEXT_LENGTH":50,          # max length of target text
  "SEED": 42                  # set seed for reproducibility 

}

In [24]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

Sun Apr 23 02:52:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    50W / 400W |    717MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
T5Trainer(dataframe=dfAll, source_text="question", target_text="answer", model_params=model_params, output_dir="./drive/MyDrive/T5")

[03:01:22] [Model]: Loading mrm8488/t5-base-finetuned-common_gen...              <ipython-input-11-91b2f7e68627>:13
                                                                                                                   

[03:01:27] [Data]: Reading data...                                               <ipython-input-11-91b2f7e68627>:24
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                source_text (Question)                  |                  target_text (Answer)                  |
|--------------------------------------------------------+--------------------------------------------------------|
|  Generate next sentence that makes reader feels joy,   |    My little sister started swinging her net at the    |
| surprise, anticipation.<extra_id_0>KEYWORD: my little  |                      butterflies.                      |
|  sister, her net, butterflies<extra_id_1>CONTEXT: My   |                                                        |
|little sister wanted to catch a butterfly for a pet.She |                                                        |
|had no idea what she was doing.I helped her build a net |                                                        |
|             and showed her where to look.              |                                                        |
|    Generate next sentence that makes reader feels      |    Sadly, she cannot drink Diet Coke while pregnant.   |
|        sadness.<extra_id_0>KEYWORD: she, diet          |                                                        |
|coke<extra_id_1>CONTEXT: Jenna loves Diet Coke.She and  |                                                        |
|          her husband decide to have a baby.            |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (28892, 2)

TRAIN Dataset: (28892, 2)

...

[07:41:12] [Initiating Validation]...                                            <ipython-input-11-91b2f7e68627>:87
                                                                                                                   

           [Validation Completed.]                                               <ipython-input-11-91b2f7e68627>:95
                                                                                                                   

[Model] Model saved @ ./drive/MyDrive/T5/model_files

[Validation] Generation on Validation data saved @ ./drive/MyDrive/T5/predictions.csv

[Logs] Logs saved @ ./drive/MyDrive/T5/logs.txt

In [ ]:
result = pd.read_csv('drive/MyDrive/T5/predictions.csv')


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=result['Generated Text'], references=result['Actual Text'])
print(results)